In [1]:
import os
import sys
sys.path.append('../..')

# import openai
# from langchain.chains import ConversationalRetrievalChain, RetrievalQA
# from langchain.chat_models import ChatOpenAI
# from langchain.document_loaders import DirectoryLoader, TextLoader
# from langchain_openai import OpenAIEmbeddings
# from langchain.indexes import VectorstoreIndexCreator
# from langchain.indexes.vectorstore import VectorStoreIndexWrapper
# from langchain.llms import OpenAI
# from langchain.vectorstores import Chroma


import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
# from langchain_chroma import Chroma
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
from py3810.myUtils import pickle_dump, pickle_load
path_lumen_docs = '..\langchain\docs\lumen\\docs\\'

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('.env\my_api_key.env')) # read local .env file

os.environ["OPENAI_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"]
os.environ["SECRET_KEY"]


'my_secret_key'

In [3]:
import textwrap

def print_wrapped(text, width=80):
  """
  Prints a long string to the console, wrapped to fit within a specified width.

  Args:
      text: The long string to be wrapped.
      width: The desired width for each line (default: 80 columns).
  """
  wrapped_text = textwrap.wrap(text, width=width)
  for line in wrapped_text:
    print(line)

# Example usage
long_string = "This is a very long string that needs to be wrapped to fit within 80 columns. It can contain spaces, punctuation, and even newlines."
print_wrapped(long_string)


This is a very long string that needs to be wrapped to fit within 80 columns. It
can contain spaces, punctuation, and even newlines.


In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [5]:
import pandas as pd


# Set the path to the directory containing the Excel file
path_lumen_docs = "../langchain/docs/lumen/docs/"
excel_filename = "lumen_sitemapXML.xlsx"

# Define a helper function to flatten a nested list
def fatten_list(nested_list):
    return [item for sublist in nested_list for item in sublist]

# Construct the file path of the Excel file
file = path_lumen_docs + excel_filename

# Extract the data from Excel file column A, first line as a header
df = pd.read_excel(file, usecols='A', header=0)

# Convert the data frame to a list
column_A = df.values.tolist()

# Create a list of unique URLs by flattening the list and removing duplicates
lumen_urls = sorted(fatten_list(column_A))

# Print the length of the lumen_urls list
print(f'len(lumen_urls): {len(lumen_urls)}')

# Print the lumen_urls list
print(lumen_urls)

len(lumen_urls): 173
['https://www.lumenoptometric.com/', 'https://www.lumenoptometric.com/about-us/', 'https://www.lumenoptometric.com/about-us/appointments-forms/', 'https://www.lumenoptometric.com/about-us/gallery/', 'https://www.lumenoptometric.com/about-us/office/', 'https://www.lumenoptometric.com/about-us/testimonials/', 'https://www.lumenoptometric.com/blog/', 'https://www.lumenoptometric.com/blog/amd-awareness-month/3-big-amd-myths-debunked/', 'https://www.lumenoptometric.com/blog/covid-19/contact-lens-wear-during-covid-19/', 'https://www.lumenoptometric.com/blog/covid-19/how-to-stop-masks-from-fogging-glasses/', 'https://www.lumenoptometric.com/blog/eye-care/10-essential-rules-for-contact-lens-wearers/', 'https://www.lumenoptometric.com/blog/eye-care/3-reasons-why-kids-with-myopia-need-to-spend-more-time-outdoors/', 'https://www.lumenoptometric.com/blog/eye-care/3-workplace-eye-wellness-tips-for-employees/', 'https://www.lumenoptometric.com/blog/eye-care/4-effective-methods-f

In [16]:
loader = WebBaseLoader(lumen_urls[0:2])
docs = loader.load()

In [17]:
docs

[Document(page_content='\n\n\n\n\n\n\n\n\n\nLumen Optometric | Sierra Madre, CA | Eye Care | Best Optometrist\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n Skip to primary navigation Skip to main content(626) 921-019914 West Sierra Madre Blvd, Sierra Madre, CA 91024Call Us(626) 921-0199Schedule An AppointmentMyopia Management\n\nWhat is myopia?\nIs myopia unhealthy?\nNature Versus Nurture\nMyopia Treatments\nOur technology\n\n\nScleral Lenses / Keratoconus\n\nPoseyedon Lens\n\n\nNeurolens Therapy\nOrthokeratology\n\nHow’s Ortho-K Work?\nIs Ortho-k Safe?\nCandidacy\nOur Technology\nFrequently Asked Questions\nAdult vs. Children’s Designs\n\n\nEye Exams\n\nContact Lens Exams\n\n\nPatient Center\n\nAppointment & Forms\nCOVID-19 Protocols\nVideos\n\n\nAbout Us\n\nOffice\nTestimonials\nBlog\n\n\nMyopia ManagementLumen Optometric specializes in slowing down the worsening of your child\'s eyes! Protect them now and the eye hea

In [10]:
# # Load, chunk and index the contents of the blog.
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [12]:
print_wrapped(rag_chain.invoke("What is their phone number?"))

To schedule an appointment at Lumen Optometric, call (626) 921-0199.


In [18]:
# https://python.langchain.com/v0.1/docs/integrations/vectorstores/chroma/
query = 'where is lumen optometric?'

persist_directory = "./chroma_db"
embedding_function = OpenAIEmbeddings()
collection_name = 'lumen_webpages_test'

# save to disk
db2 = Chroma.from_documents(
  documents=docs,
  embedding=embedding_function,
  collection_name=collection_name,
  persist_directory=persist_directory
  )
ans = db2.similarity_search(query)
# print(docs[0].page_content)
ans

# # load from disk
# db3 = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)
# docs = db3.similarity_search(query)
# print(docs[0].page_content)

[Document(page_content='\n\n\n\n\n\n\n\n\n\nLumen Optometric | Sierra Madre, CA | Eye Care | Best Optometrist\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n Skip to primary navigation Skip to main content(626) 921-019914 West Sierra Madre Blvd, Sierra Madre, CA 91024Call Us(626) 921-0199Schedule An AppointmentMyopia Management\n\nWhat is myopia?\nIs myopia unhealthy?\nNature Versus Nurture\nMyopia Treatments\nOur technology\n\n\nScleral Lenses / Keratoconus\n\nPoseyedon Lens\n\n\nNeurolens Therapy\nOrthokeratology\n\nHow’s Ortho-K Work?\nIs Ortho-k Safe?\nCandidacy\nOur Technology\nFrequently Asked Questions\nAdult vs. Children’s Designs\n\n\nEye Exams\n\nContact Lens Exams\n\n\nPatient Center\n\nAppointment & Forms\nCOVID-19 Protocols\nVideos\n\n\nAbout Us\n\nOffice\nTestimonials\nBlog\n\n\nMyopia ManagementLumen Optometric specializes in slowing down the worsening of your child\'s eyes! Protect them now and the eye hea

In [11]:
print_wrapped(rag_chain.invoke("What is Task Decomposition?"))

Task Decomposition is a technique where complex tasks are broken down into
smaller and simpler steps to enhance model performance. This process involves
transforming big tasks into manageable tasks through steps like Chain of Thought
or Tree of Thoughts. Task decomposition can be achieved through simple
prompting, task-specific instructions, or human inputs.


In [15]:
# cleanup
vectorstore.delete_collection()